In [2]:
# 1、keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models, layers, utils
import matplotlib.pylab as plt
import cv2, os

# 加载数据
(train_img, train_label), (test_img, test_label) = mnist.load_data()

# 查看测试集中数据
img1 = test_img[0]
plt.imshow(img1, cmap=plt.cm.binary)
plt.show()

# 建立一个空神经网络模型
network = models.Sequential()
# 增加隐藏层和输出层
network.add(layers.Dense(512, activation='relu', input_shape=(28*28, )))
network.add(layers.Dense(10, activation='softmax'))

# 将构建好的网络进行编译
# categorical_crossentropy 交叉熵
# metrics 判断正确式的方式      accuracy：正确率
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# 进行归一化
train_img = train_img.reshape((60000, 28*28))
train_img = train_img.astype('float32') / 255

test_img = test_img.reshape((10000, 28*28))
test_img = test_img.astype('float32') / 255

# 进行one hot编码
train_label = utils.to_categorical(train_label)
test_label = utils.to_categorical(test_label)

# 训练
network.fit(train_img, train_label, epochs=5, batch_size=128)

# 测试数据输入后识别效果
test_loss, test_acc = network.evaluate(test_img, test_label, verbose=1)

# 输入一张图片，查看识别效果
img_list = os.listdir('../../imgs/digit_shouxie')
# zhenshi = [6, 2, 3, 3, 6, 4, 5]
for i in range(len(img_list)):
    digit = cv2.imread(f'../../imgs/digit_shouxie/{img_list[i]}', 0)
    # digit = cv2.resize(digit, (28, 28), interpolation=cv2.INTER_CUBIC)
    plt.imshow(digit, cmap=plt.cm.binary)
    plt.show()
    digit = digit.reshape((1, 28*28))
    res = network.predict(digit)
    print(res)
    print(res.shape)
    print(res.shape[1])
    for j in range(res.shape[1]):
        if (res[0][j] == 1):
            print("the number for the picture is : ", j)
            break

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# 2、NeuralNetWork
import numpy as np
import scipy.special


class NeuralNetWork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        """
        设置初始化
        :param inputnodes: 输入层
        :param hiddennodes: 中间层
        :param outputnodes: 输出层
        :param learningrate: 学习率
        """
        self.inputnodes = inputnodes
        self.hiddennodes = hiddennodes
        self.outputnodes = outputnodes
        self.learningrate = learningrate

        # 初始化权重矩阵,wih表示输入层中和中间层之间的权重矩阵，who表示中间层和输出层之间的权重矩阵
        self.wih = np.random.rand(self.hiddennodes, self.inputnodes) - 0.5
        self.who = np.random.rand(self.outputnodes, self.hiddennodes) - 0.5
        print(self.wih)
        print(self.who)

        # 设置激活函数
        self.activation_function = lambda x: scipy.special.expit(x)

    def train(self, inputs_list, targets_list):
        """
        根据输入的训练数据更新节点链路权重
        :param inputs_list: 输入的训练数据
        :param targets_list:    训练数据对应的正确结果
        :return:
        """
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        # 计算从输入层到中间层的信号量
        hidden_inputs = np.dot(self.wih, inputs)
        # 进行激活函数
        hidden_outputs = self.activation_function(hidden_inputs)
        # 计算从中间层到输出层的信号量
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)

        # 计算误差
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors * final_outputs * (1 - final_outputs))


        self.who += self.learningrate * np.dot((output_errors * final_outputs * (1 - final_outputs)),
                                               np.transpose(hidden_outputs))

        self.wih += self.learningrate * np.dot((hidden_errors * hidden_outputs * (1 - hidden_outputs)),
                                               np.transpose(inputs))
        pass

    def query(self, inputs):
        # 计算从输入层到中间层的信号量
        hidden_inputs = np.dot(self.wih, inputs)
        # 进行激活函数
        hidden_outputs = self.activation_function(hidden_inputs)
        # 计算从中间层到输出层的信号量
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs


if __name__ == '__main__':
    # 初始化网络
    input_nodes = 784
    hidden_nodes = 100
    output_nodes = 10
    learning_rate = 0.1
    n = NeuralNetWork(input_nodes, hidden_nodes, output_nodes, learning_rate)
    # 读取图片 csv中每一行代表一个图片的像素
    training_data_file = open("../../imgs/digit_shouxie/mnist_train.csv", 'r')
    training_data_list = training_data_file.readlines()
    training_data_file.close()
    # 设置网络的训练循环次数
    epochs = 5
    for e in range(epochs):
        # 把数据依靠','区分，并分别读入
        for record in training_data_list:
            all_values = record.split(',')
            inputs = (np.asfarray(all_values[1:])) / 255.0 * 0.99 + 0.01
            # 设置图片与数值的对应关系
            targets = np.zeros(output_nodes) + 0.01
            targets[int(all_values[0])] = 0.99
            n.train(inputs, targets)
    test_data_file = open("../../imgs/digit_shouxie/mnist_test.csv")
    test_data_list = test_data_file.readlines()
    test_data_file.close()
    scores = []
    for record in test_data_list:
        all_values = record.split(',')
        correct_number = int(all_values[0])
        print("该图片对应的数字为:", correct_number)
        # 预处理数字图片
        inputs = (np.asfarray(all_values[1:])) / 255.0 * 0.99 + 0.01
        # 让网络判断图片对应的数字
        outputs = n.query(inputs)
        # 找到数值最大的神经元对应的编号
        label = np.argmax(outputs)
        print("网络认为图片的数字是：", label)
        if label == correct_number:
            scores.append(1)
        else:
            scores.append(0)
    print(scores)
    # 计算图片判断的成功率
    scores_array = np.asarray(scores)
    print("perfermance = ", scores_array.sum() / scores_array.size)

In [ ]:
# 3、tensorflow
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
 
#使用numpy生成200个随机点
x_data=np.linspace(-0.5,0.5,200)[:,np.newaxis]
noise=np.random.normal(0,0.02,x_data.shape)
y_data=np.square(x_data)+noise
 
#定义两个placeholder存放输入数据
x=tf.placeholder(tf.float32,[None,1])
y=tf.placeholder(tf.float32,[None,1])
 
#定义神经网络中间层
Weights_L1=tf.Variable(tf.random_normal([1,10]))
biases_L1=tf.Variable(tf.zeros([1,10]))    #加入偏置项
Wx_plus_b_L1=tf.matmul(x,Weights_L1)+biases_L1
L1=tf.nn.tanh(Wx_plus_b_L1)   #加入激活函数

#定义神经网络输出层
Weights_L2=tf.Variable(tf.random_normal([10,1]))
biases_L2=tf.Variable(tf.zeros([1,1]))  #加入偏置项
Wx_plus_b_L2=tf.matmul(L1,Weights_L2)+biases_L2
prediction=tf.nn.tanh(Wx_plus_b_L2)   #加入激活函数

#定义损失函数（均方差函数）
loss=tf.reduce_mean(tf.square(y-prediction))
#定义反向传播算法（使用梯度下降算法训练）
train_step=tf.train.GradientDescentOptimizer(0.1).minimize(loss)
 
with tf.Session() as sess:
    #变量初始化
    sess.run(tf.global_variables_initializer())
    #训练2000次
    for i in range(2000):
        sess.run(train_step,feed_dict={x:x_data,y:y_data})
 
    #获得预测值
    prediction_value=sess.run(prediction,feed_dict={x:x_data})
 
    #画图
    plt.figure()
    plt.scatter(x_data,y_data)   #散点是真实值
    plt.plot(x_data,prediction_value,'r-',lw=5)   #曲线是预测值
    plt.show()
